# Super simple Kubeflow Pipelines

Here we will schedule a one pipeline that will download artifacts from minio buckets

In [7]:
# bootstrap KFP
!pip3 install --upgrade 'https://storage.googleapis.com/ml-pipeline/release/0.1.4/kfp.tar.gz'

  Using cached https://storage.googleapis.com/ml-pipeline/release/0.1.4/kfp.tar.gz
  Running setup.py bdist_wheel for kfp ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-yms5qewy/wheels/fb/e7/7d/37ec9ec6d9b68fd5421789b973aa0048f2554b78fa9dedfb26
Successfully built kfp
  Found existing installation: kfp 0.1
    Uninstalling kfp-0.1:
      Successfully uninstalled kfp-0.1


## Environment setup

In [8]:
# our minio credentials
S3_ENDPOINT = 'minio.app.black.antondemo01.dev.superhub.io'
S3_ACCESS_KEY = 'admin'
S3_SECRET_KEY = 'password'

%env MC_HOSTS_minio='https://{}:{}@{}'.format(S3_ACCESS_KEY, S3_SECRET_KEY, S3_ENDPOINT)

## Define a Pipeline

In [9]:
import kfp.dsl as dsl
from kubernetes import client as k8sc

@dsl.pipeline(
  name='Super simple minio integration',
  description='I as a pipeline want to read a file from minio bucket'
)
def hello_minio_pipeline(filename: dsl.PipelineParam):
  op1 = dsl.ContainerOp(
     name='download',
     image='minio/mc',
     command=['mc', '--no-color'],
     arguments=['cp', 'minio/%s' % filename, '/tmp/results.txt'],
     file_outputs={'downloaded': '/tmp/results.txt'}
  ).add_env_variable(
        k8sc.V1EnvVar(
            name='MC_HOSTS_minio', 
            value='https://{}:{}@{}'.format(S3_ACCESS_KEY, S3_SECRET_KEY, S3_ENDPOINT), 
        ))
  op2 = dsl.ContainerOp(
     name='echo',
     image='library/bash:4.4.23',
     command=['sh', '-c'],
     arguments=['echo %s' % op1.output])

### Define and run and experiment 

Code below will create a new experiement **Hello Minio** and run it

In [10]:
import kfp
import kfp.compiler as compiler

def get_or_create_experiement(client, name):
    next_page_token = ''
    while next_page_token is not None:
      resp = client.list_experiments(page_size=100, page_token=next_page_token)
      next_page_token = resp.next_page_token
      for e in resp.experiments:
        if e.name == name:
          return client.get_experiment(e.id)
    return client.create_experiment(name=name)
    

client = kfp.Client()
exp = get_or_create_experiement(client, 'Hello minio')
compiler.Compiler().compile(hello_minio_pipeline, 'pipeline.tar.gz')

run = client.run_pipeline(exp.id, 
                          'pipeline 1', 
                          'pipeline.tar.gz',
                          params={'filename': 'default/hello.txt'})
